In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!tar xvzf runs.tar.gz

runs/
runs/Jun09_18-54-32_44eacb9e064a/
runs/Jun09_18-54-32_44eacb9e064a/Accuracy_val/
runs/Jun09_18-54-32_44eacb9e064a/Accuracy_val/events.out.tfevents.1654801127.44eacb9e064a.8648.3
runs/Jun09_18-54-32_44eacb9e064a/Accuracy_train/
runs/Jun09_18-54-32_44eacb9e064a/Accuracy_train/events.out.tfevents.1654801126.44eacb9e064a.8648.2
runs/Jun09_18-54-32_44eacb9e064a/Loss_train/
runs/Jun09_18-54-32_44eacb9e064a/Loss_train/events.out.tfevents.1654801126.44eacb9e064a.8648.1
runs/Jun09_18-54-32_44eacb9e064a/events.out.tfevents.1654800874.44eacb9e064a.8648.0


In [4]:


from torch.optim.lr_scheduler import LambdaLR
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import os
import csv
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split
import math
#set random seeds
seed=42
torch.manual_seed(seed)
writer = SummaryWriter()
# Device configuration
device = torch.device('cuda')
# Hyper-parameters 
# input_size = 784 # 28x28
num_classes = 1
num_epochs = 300
batch_size = 32
learning_rate = 0.001
input_size = 128
sequence_length = 300
hidden_size = 3500
num_layers = 2

# test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
#                                           batch_size=batch_size, 
#                                           shuffle=False)
org_path = os.getcwd()
#read csv
rows = []
rows_test = []
with open("/content/bc_detection_train.csv", 'r') as file:
    csvreader = csv.reader(file)
    header = next(csvreader)
    for row_1 in csvreader:
        rows.append(row_1)
print(header)
print(rows[:][0])
with open("/content/bc_detection_val.csv", 'r') as file:
    csvreader = csv.reader(file)
    header = next(csvreader)
    for row_2 in csvreader:
        rows_test.append(row_2)
print(header)
print(rows_test[:][0])

#reading input data
#path = os.getcwd() + '/' + 'resnet50'
path = '/content/drive/MyDrive/audio_features_train_latest'
dir_list = os.listdir(path)
os.chdir(path)

combined_data = np.array([np.load(fname[0]+'_audio_vggish.npy') for fname in rows])
#combined_data = np.array([print(fname[0]+'_audio_vggish.npy') if ((np.load(fname[0]+'_audio_vggish.npy').shape) != (300,128)) else 0 for fname in rows ])
labels = np.array([np.array(fname[1], dtype=np.float16) for fname in rows])
path = '/content/drive/MyDrive/audio_features_val'
dir_list = os.listdir(path)
os.chdir(path)
combined_data_test = np.array([np.load(fname[0]+'_audio_vggish.npy') for fname in rows_test])
#combined_data_test = np.array([print(fname[0]+'_audio_vggish.npy') if ((np.load(fname[0]+'_audio_vggish.npy').shape) != (300,128)) else 0 for fname in rows_test ])
labels_test = np.array([np.array(fname[1], dtype=np.float16) for fname in rows_test])
os.chdir(org_path)

tensor_x = torch.Tensor(combined_data) # transform to torch tensor
tensor_y = torch.Tensor(labels.astype(np.float))

tensor_x_test = torch.Tensor(combined_data_test) # transform to torch tensor
tensor_y_test = torch.Tensor(labels_test.astype(np.float))
my_dataset = TensorDataset(tensor_x,tensor_y) # create your datset
dataset_train, dataset_validate = train_test_split(
        my_dataset, test_size=0.02, random_state=84
    )
my_dataset_test = TensorDataset(tensor_x_test,tensor_y_test)
print(tensor_x.shape)
print(tensor_y.shape)
print(tensor_x_test.shape)
print(tensor_y_test.shape)
my_dataloader = DataLoader(dataset_train,batch_size=batch_size, shuffle = True) # create your dataloader
my_dataloader_val = DataLoader(dataset_validate,batch_size=batch_size) 
my_dataloader_test = DataLoader(my_dataset_test,batch_size=batch_size) 


def get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps,
    num_training_steps,
    num_cycles=7.0 / 16.0,
    last_epoch=-1,
):
    def _lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        no_progress = float(current_step - num_warmup_steps) / float(
            max(1, num_training_steps - num_warmup_steps)
        )
        return max(0.0, math.cos(math.pi * num_cycles * no_progress))
    return LambdaLR(optimizer, _lr_lambda, last_epoch)

# for i, (images, labels) in enumerate(my_dataloader): 
#   print(images.shape)
#   print(labels)
#Fully connected neural network with one hidden layer
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        #self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        # -> x needs to be: (batch_size, seq, input_size)
        
        # or:
        #self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.lynm = nn.LayerNorm(input_size)
        self.lynm1 = nn.LayerNorm(hidden_size)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        # Set initial hidden states (and cell states for LSTM)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        h0 = torch.nn.init.xavier_normal_(h0).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.nn.init.xavier_normal_(c0).to(device)
        
        # x: (n, 28, 28), h0: (2, n, 128)
        
        # Forward propagate RNN
        #out, _ = self.rnn(x, h0)  
        # or:
        x= self.lynm(x)
        out, _ = self.lstm(x, (h0,c0))  
        
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        # out: (n, 28, 128)
        # print("out bef")
        # print(out)
        # Decode the hidden state of the last time step
        out = out[:, -1, :]

       
        # out: (n, 128)
        x= self.lynm1(out) 
        out = self.fc(x)

        # print("out aft")
        # print(out)
        # out: (n, 10)
        return torch.sigmoid(out)
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)
# Loss and optimizer

# for name, param in model.named_parameters():
#   if 'bias' in name:
#      nn.init.constant(param, 0.0)
#   elif 'weight' in name:
#      nn.init.xavier_normal(param)

def count_parameters(model):

 return sum(p.numel() for p in model.parameters() if p.requires_grad)

s = count_parameters(model)

print(s)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay= 0.0005)
#optimizer = torch.optim.SGD(model.parameters(), lr= learning_rate, momentum=0.9)  
scheduler = get_cosine_schedule_with_warmup(
        optimizer, 500, 402660
    )
# Train the model
n_total_steps = len(my_dataloader)
best_val_acc = 0
for epoch in range(num_epochs):
    correct = 0
    num_samples = 0
    model.train()
    for i, (images, labels) in enumerate(my_dataloader):
        # origin shape: [N, 1, 28, 28]
        # resized: [N, 300, 2048][N,300,128]
        
        images = images.reshape(-1, sequence_length, input_size).to(device)
        # print(images.shape)
        labels = labels.to(device)
        num_samples+=labels.size(0)
        # Forward pass
        outputs = model(images)
        # print(f'outputs.shape:{outputs.shape}')
        # print(labels.shape)
        # outputs = outputs.squeeze()
        loss = criterion(outputs, labels.unsqueeze(1))
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        predicted = (outputs > 0.5).long()
        # print(f'predicted.shape:{predicted.shape}')
        # print(f'labels.shape:{labels.shape}')
        # print(f'correct_pre:{correct}')
        correct += (predicted.squeeze()== labels).sum().item()
        # print(f'predicted:{predicted}')
        # print(f'correct:{correct}')
        # print(f'labels.size:{labels.size(0)}')
        # if (i+1) % 1 == 0:
        #     print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
        # print(f'correct:{correct}')
    # print(f'tensor_y.size(0):{tensor_y.size(0)}')
    print('[%d/%d] loss: %f, accuracy: %.3f' %
          (i , epoch, loss.item(), 100 * correct /num_samples))
    writer.add_scalars('Loss',{'train':loss.item()},epoch)
    writer.add_scalars('Accuracy', {'train': 100 * correct /num_samples},epoch)
        
    # Test the model
    # In test phase, we don't need to compute gradients (for memory efficiency)
    num_samples_val = 0
    model.eval()
    with torch.no_grad():
        correct_val = 0
        for i, (images, labels) in enumerate(my_dataloader_val):
            images = images.reshape(-1, sequence_length, input_size).to(device)
            labels = labels.to(device)
            num_samples_val+=labels.size(0)
            outputs = model(images)
            loss = criterion(outputs, labels.unsqueeze(1))
            predicted = (outputs > 0.5).long()
            correct_val += (predicted.squeeze()== labels).sum().item()
        val_acc = 100 * correct_val / num_samples_val
        print(f'Accuracy of the network on the validation: {val_acc} %')
        print(f'loss of the network on the validation: {loss} ')
        writer.add_scalars('loss', {'val': loss},epoch)
    if(val_acc> best_val_acc):
        best_val_acc = val_acc
        torch.save(model.state_dict(),'./best_model'+'_'+str(learning_rate)+'_'+str(num_layers)+'.ckpt')                         
        print("best model with val acc "+ str(best_val_acc)+ "is saved")
model.eval()
model.load_state_dict(torch.load('/content/best_model'+'_'+str(learning_rate)+'_'+str(num_layers)+'.ckpt'))   
with torch.no_grad():
        correct_val = 0
        num_samples_val = 0
        for i, (images, labels) in enumerate(my_dataloader_test):
            images = images.reshape(-1, sequence_length, input_size).to(device)
            labels = labels.to(device)
            num_samples_val+=labels.size(0)
            outputs = model(images)
            predicted = (outputs > 0.5).long()
            correct_val += (predicted.squeeze()== labels).sum().item()
        val_acc = 100 * correct_val / num_samples_val
        print(f'Accuracy of the network on the test: {val_acc} %')

['id', 'label']
['0_train_rec18_pos3', '1']
['id', 'label']
['6716_val_rec27_pos4', '1']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:72: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:75: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


torch.Size([6711, 300, 128])
torch.Size([6711])
torch.Size([2853, 300, 128])
torch.Size([2853])
148858757
[205/0] loss: 0.730956, accuracy: 50.061
Accuracy of the network on the validation: 54.81481481481482 %
loss of the network on the validation: 0.7270519137382507 
best model with val acc 54.81481481481482is saved
[205/1] loss: 0.739901, accuracy: 48.875
Accuracy of the network on the validation: 54.074074074074076 %
loss of the network on the validation: 0.7630759477615356 


KeyboardInterrupt: ignored

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
!tar cvzf runs.tar.gz runs

runs/
runs/Jun10_18-36-20_ef20d790990e/
runs/Jun10_18-36-20_ef20d790990e/Loss_train/
runs/Jun10_18-36-20_ef20d790990e/Loss_train/events.out.tfevents.1654886780.ef20d790990e.81.2
runs/Jun10_18-36-20_ef20d790990e/Accuracy_val/
runs/Jun10_18-36-20_ef20d790990e/Accuracy_val/events.out.tfevents.1654886784.ef20d790990e.81.4
runs/Jun10_18-36-20_ef20d790990e/Accuracy_train/
runs/Jun10_18-36-20_ef20d790990e/Accuracy_train/events.out.tfevents.1654886780.ef20d790990e.81.3
runs/Jun10_18-36-20_ef20d790990e/events.out.tfevents.1654886180.ef20d790990e.81.1
runs/Jun09_18-54-32_44eacb9e064a/
runs/Jun09_18-54-32_44eacb9e064a/Loss_train/
runs/Jun09_18-54-32_44eacb9e064a/Loss_train/events.out.tfevents.1654801126.44eacb9e064a.8648.1
runs/Jun09_18-54-32_44eacb9e064a/Accuracy_val/
runs/Jun09_18-54-32_44eacb9e064a/Accuracy_val/events.out.tfevents.1654801127.44eacb9e064a.8648.3
runs/Jun09_18-54-32_44eacb9e064a/Accuracy_train/
runs/Jun09_18-54-32_44eacb9e064a/Accuracy_train/events.out.tfevents.1654801126.44eacb